In [1]:
from cdp_data import CDPInstances, datasets

seattle_df = datasets.get_session_dataset(
    CDPInstances.Seattle,
    start_datetime="2023-01-01",
    end_datetime="2023-05-01",
    store_transcript=True,
    store_transcript_as_csv=True,
)
seattle_df.sample(3)

/Users/eva/miniconda3/envs/minutes-item-seg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Converting transcripts: 100%|██████████| 32/32 [00:17<00:00,  1.87it/s]


,session_datetime,session_index,session_content_hash,video_uri,video_start_time,video_end_time,caption_uri,external_source_id,id,key,event,transcript,transcript_path,transcript_as_csv_path
30,2023-04-26 16:30:00+00:00,0,62c9cf87d917c0cb8ac36a85218950129cb584af937639...,https://video.seattle.gov/media/council/econ_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,c726d2ef6b06,session/c726d2ef6b06,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/eva/active/cdp/minutes-item-seg/cdp-dat...,/Users/eva/active/cdp/minutes-item-seg/cdp-dat...
25,2023-04-12 16:30:00+00:00,0,527dffa11193464222d35dc7949005c3475c0c09b4e006...,https://video.seattle.gov/media/council/econ_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,c460a8868bbf,session/c460a8868bbf,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/eva/active/cdp/minutes-item-seg/cdp-dat...,/Users/eva/active/cdp/minutes-item-seg/cdp-dat...
15,2023-03-08 17:30:00+00:00,0,141965d5128e4f840ea70e064504c0704302002923ae02...,https://video.seattle.gov/media/council/econ_0...,None,None,https://www.seattlechannel.org/documents/seatt...,None,48fe292ca91b,session/48fe292ca91b,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/eva/active/cdp/minutes-item-seg/cdp-dat...,/Users/eva/active/cdp/minutes-item-seg/cdp-dat...


In [2]:
from functools import partial
import pandas as pd

def _process_session(row: pd.Series, extra_meta: dict) -> dict:
    # Read transcript CSV
    transcript = pd.read_csv(row.transcript_as_csv_path)
    
    return {
        "text": " ".join(transcript.text),
        "meta": {
            "event_id": row.event.id,
            "session_id": row.id,
            **extra_meta,
        },
    }

# Create partial with muni name
process_session_seattle = partial(_process_session, extra_meta={"muni": "seattle"})

# Process all rows in dataset to get full text to a single column
processed_sessions = pd.DataFrame(
    list(seattle_df.apply(process_session_seattle, axis=1)), 
)
processed_sessions.sample(1)

,text,meta
5,Thank you. Thank you very much. Good morning. ...,"{'event_id': 'a771898d7666', 'session_id': '82..."


In [3]:
# Store processed dataset to JSONL
processed_sessions.to_json("seattle.jsonl", orient="records", lines=True)